In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/IR_Project/Scholar/

/content/drive/.shortcut-targets-by-id/1qAY9sRpD8XGe6_oa-jCd9NWhJPZ_HOwg/IR_Project/Scholar


In [ ]:
import json
import pandas as pd
from pprint import pprint
from tqdm import tqdm

### download data

In [ ]:
!curl -O https://ciir.cs.umass.edu/downloads/LaMP/LaMP_5/train/train_questions.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  858M  100  858M    0     0  11.5M      0  0:01:14  0:01:14 --:--:-- 11.6M


In [ ]:
!curl -O https://ciir.cs.umass.edu/downloads/LaMP/LaMP_5/train/train_outputs.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1006k  100 1006k    0     0   883k      0  0:00:01  0:00:01 --:--:--  883k


## Data

In [ ]:
with open('train_questions.json') as f:
    d = json.load(f)
data = pd.DataFrame.from_dict(d)

with open('train_outputs.json') as f:
    t = json.load(f)

outputs=pd.DataFrame.from_dict(t['golds'])
final_data=pd.merge(data, outputs, on='id')

In [ ]:
final_data.head(1)

,id,input,profile,output
0,400,Generate a title for the following abstract of...,[{'title': 'Convex Drawings of Internally Tric...,Partitioning Trees of Supply and Demand


In [ ]:
#making my testing sample
test_data=final_data.sample(100)

# STEP1: Retriever

### bm25

In [ ]:
!pip install langchain
!pip install rank_bm25

In [ ]:
from langchain.schema import Document
from langchain.retrievers import BM25Retriever

In [ ]:
def get_relevant_docs(query : str, docs: list):
  documents =  []

  for item in docs:
      page = Document(page_content=item['abstract'],metadata = {"id" : item['id'], "title" : item['title']})
      documents.append(page)

  retriever = BM25Retriever.from_documents(documents)
  top_docs=retriever.get_relevant_documents(query)[:3]
  top3=[]
  for i in top_docs[:3]:
    res_dict={'abstract':i.page_content,'title':i.metadata['title'],'id':i.metadata['id']}
    top3.append(res_dict)
  return top3

In [ ]:
retriever_results=[]
for row in tqdm(range(test_data.shape[0])):
  input_text = test_data.iloc[row]['input']
  top_ret=get_relevant_docs(input_text, test_data.iloc[row]['profile'])
  retriever_results.append(top_ret)

100%|██████████| 100/100 [00:01<00:00, 51.34it/s]


In [ ]:
test_data['bm25_results']=retriever_results

In [ ]:
test_data.head()

,id,input,profile,output,bm25_results
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,[{'abstract': 'Industrial Control Systems (ICS...
2242,402242,Generate a title for the following abstract of...,"[{'title': 'On updating signal subspaces', 'ab...",Hands-On Training for Undergraduates in High-P...,[{'abstract': 'Parallel programming of high-pe...
2577,402577,Generate a title for the following abstract of...,[{'title': 'A Note on Filtration and Granular ...,On possible rules and apriori algorithm in non...,[{'abstract': 'Minimal rule generation in Non-...
1399,401399,Generate a title for the following abstract of...,[{'title': 'Temporal representation for scient...,Framework for bringing data streams to the grid,[{'abstract': 'The dQUOB system satisfies clie...
7707,407707,Generate a title for the following abstract of...,[{'title': 'Completeness In Differential Appro...,Multiparameterizations for max $k$-set cover a...,[{'abstract': 'We study the parameterized comp...


### contriever

In [ ]:
## imporintng contriever results run on local

In [ ]:
test_data=pd.read_pickle('contriever/cont_res_scholar_100.pkl')

In [ ]:
test_data.head()

,id,input,profile,output,input_keywords,input_summary,contriever
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,"automata, swat, classifier, detection, anomaly",the problem of process detection in water tank...,[{'title': 'WaterJam: An Experimental Case Stu...
2242,402242,Generate a title for the following abstract of...,"[{'title': 'On updating signal subspaces', 'ab...",Hands-On Training for Undergraduates in High-P...,"java, programming, computing, computational, p...",this paper describes a practical computing cou...,[{'title': 'Parallel programming in computatio...
2577,402577,Generate a title for the following abstract of...,[{'title': 'A Note on Filtration and Granular ...,On possible rules and apriori algorithm in non...,"algorithm, minimalcertainrules, deterministici...","in this paper, we introduce a framework of gen...",[{'title': 'Discernibility functions and minim...
1399,401399,Generate a title for the following abstract of...,[{'title': 'Temporal representation for scient...,Framework for bringing data streams to the grid,"streams, stream, streaming, grid, data",this article introduces a model for bringing e...,[{'title': 'Calder Query Grid Service: Insight...
7707,407707,Generate a title for the following abstract of...,[{'title': 'Completeness In Differential Appro...,Multiparameterizations for max $k$-set cover a...,"parameterizations, complexity, parameterized, ...",study the complexity of several parameterizati...,[{'title': 'Multi-parameter Analysis for Local...


# KEYWORD EXTRACTION

In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=9806256dbc3a2acbdc1798a823ee4353fdc3b50b9582071b0d4efce3abed232d
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=5b4b6afd49f484d20c336eba0f2dc1623ad810736d10b1fd1223ac99fde810f1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
keyword_results=[]
for row in tqdm(range(test_data.shape[0])):
  #cur_list = test_data.iloc[row]['bm25_results'] #ifbm25 run this
  cur_list = test_data.iloc[row]['contriever']
  updated_list = [{'text': entry['abstract'], 'title': entry['title'], 'id': entry['id'],'keywords': kw_model.extract_keywords(entry['abstract'],, stop_words='english',use_mmr=True, diversity=0.3)} for entry in cur_list]

  #updated_list = [{'text': entry['abstract'], 'title': entry['title'], 'id': entry['id'],'keywords': kw_model.extract_keywords(entry['abstract'])} for entry in cur_list]
  keyword_results.append(updated_list)

100%|██████████| 100/100 [02:05<00:00,  1.25s/it]


In [ ]:
test_data['keyword_results']=keyword_results

In [ ]:
test_data.head()

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,"automata, swat, classifier, detection, anomaly",the problem of process detection in water tank...,[{'title': 'WaterJam: An Experimental Case Stu...,[{'text': 'An experiment was conducted to inve...
2242,402242,Generate a title for the following abstract of...,"[{'title': 'On updating signal subspaces', 'ab...",Hands-On Training for Undergraduates in High-P...,"java, programming, computing, computational, p...",this paper describes a practical computing cou...,[{'title': 'Parallel programming in computatio...,[{'text': 'Parallel programming of high-perfor...
2577,402577,Generate a title for the following abstract of...,[{'title': 'A Note on Filtration and Granular ...,On possible rules and apriori algorithm in non...,"algorithm, minimalcertainrules, deterministici...","in this paper, we introduce a framework of gen...",[{'title': 'Discernibility functions and minim...,[{'text': 'Minimal rule generation in Non-dete...
1399,401399,Generate a title for the following abstract of...,[{'title': 'Temporal representation for scient...,Framework for bringing data streams to the grid,"streams, stream, streaming, grid, data",this article introduces a model for bringing e...,[{'title': 'Calder Query Grid Service: Insight...,[{'text': 'We have architected and evaluated a...
7707,407707,Generate a title for the following abstract of...,[{'title': 'Completeness In Differential Appro...,Multiparameterizations for max $k$-set cover a...,"parameterizations, complexity, parameterized, ...",study the complexity of several parameterizati...,[{'title': 'Multi-parameter Analysis for Local...,[{'text': 'We study the parameterized complexi...


In [ ]:
test_data.to_pickle('contriever/test_data_100.pkl')

# SUMMARIZER

In [ ]:
test_data=pd.read_pickle('contriever/test_data_100.pkl')

In [ ]:
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00


In [ ]:
# Make sure you have git-lfs installed (https://git-lfs.com)
!git lfs install
!git clone https://huggingface.co/google/bigbird-pegasus-large-arxiv

Git LFS initialized.
Cloning into 'bigbird-pegasus-large-arxiv'...
remote: Enumerating objects: 50, done.
remote: Total 50 (delta 0), reused 0 (delta 0), pack-reused 50
Unpacking objects: 100% (50/50), 1.27 MiB | 1.74 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/IR_Project/Scholar/bigbird-pegasus-large-arxiv/.git/hooks/post-checkout': Permission denied


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Ensure CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to your downloaded model
model_path = "bigbird-pegasus-large-arxiv/"
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device).half()

In [ ]:
# TEST
raw_document = "Industrial Control Systems (ICS) are found in critical infrastructure such as for power generation and water treatment. When security requirements are incorporated into an ICS, one needs to test the additional code and devices added do improve the prevention and detection of cyber attacks. Conducting such tests in legacy systems is a challenge due to the high availability requirement. An approach using Timed Automata (TA) is proposed to overcome this challenge. This approach enables assessment of the effectiveness of an attack detection method based on process invariants. The approach has been demonstrated in a case study on one stage of a 6- stage operational water treatment plant. The model constructed captured the interactions among components in the selected stage. In addition, a set of attacks, attack detection mechanisms, and security specifications were also modeled using TA. These TA models were conjoined into a network and implemented in UPPAAL. The models so implemented were found effective in detecting the attacks considered. The study suggests the use of TA as an effective tool to model an ICS and study its attack detection mechanisms as a complement to doing so in a real plant-operational or under design."

# Encode the text, ensuring that it is truncated to the maximum length acceptable by the model
input_ids = tokenizer.encode(raw_document, return_tensors="pt", truncation=True, max_length=model.config.max_position_embeddings).to(device)


# Perform inference
generation_config = {
    "_from_model_config": True,
    "bos_token_id": 2,
    "decoder_start_token_id": 2,
    "eos_token_id": 1,
    "length_penalty": 0.8,
    "max_length": 256,
    "num_beams": 5,
    "pad_token_id": 0,
    "transformers_version": "4.27.0.dev0"
}



# # Perform inference
# with torch.no_grad():
#     output = model.generate(
#         input_ids,
#         num_beams=5,
#         min_length=5,
#         no_repeat_ngram_size=3,
#         max_length=50
#     )

with torch.no_grad():
    output = model.generate(
        input_ids,
        num_beams=generation_config["num_beams"],
        min_length=5,
        no_repeat_ngram_size=3,
        max_length=generation_config["max_length"],
        bos_token_id=generation_config["bos_token_id"],
        eos_token_id=generation_config["eos_token_id"],
        length_penalty=generation_config["length_penalty"],
        pad_token_id=generation_config["pad_token_id"]
    )

# Decode and print the output
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

Attention type 'block_sparse' is not possible if sequence_length: 225 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


a case study on one stage of a 6- stage operational water plant treatment was performed.<n> the attack mechanisms were modeled into and security mechanisms were implemented in so were found effective in detecting the attacks.


In [ ]:
def get_summ(raw_document):
  # Encode the text, ensuring that it is truncated to the maximum length acceptable by the model
  input_ids = tokenizer.encode(raw_document, return_tensors="pt", truncation=True, max_length=model.config.max_position_embeddings).to(device)
  # Perform inference
  generation_config = {
      "_from_model_config": True,
      "bos_token_id": 2,
      "decoder_start_token_id": 2,
      "eos_token_id": 1,
      "length_penalty": 0.8,
      "max_length": 256,
      "num_beams": 5,
      "pad_token_id": 0,
      "transformers_version": "4.27.0.dev0"
  }

  with torch.no_grad():
    output = model.generate(
        input_ids,
        num_beams=generation_config["num_beams"],
        min_length=5,
        no_repeat_ngram_size=3,
        max_length=generation_config["max_length"],
        bos_token_id=generation_config["bos_token_id"],
        eos_token_id=generation_config["eos_token_id"],
        length_penalty=generation_config["length_penalty"],
        pad_token_id=generation_config["pad_token_id"]
    )

  # Decode and print the output
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [ ]:
import pickle

In [ ]:
summ_results=[]
for row in tqdm(range(test_data.shape[0])):
  cur_list = test_data.iloc[row]['keyword_results']
  updated_list = [{'text': entry['text'], 'title': entry['title'], 'id': entry['id'],'keywords': entry['keywords'],'summary':get_summ(entry['text'])} for entry in cur_list]
  summ_results.append(updated_list)
  if(row%10==0):
    # Save to a pickle file
    with open("contriever/data_summ.pkl", "wb") as file:
      pickle.dump(summ_results, file)

100%|██████████| 100/100 [18:07<00:00, 10.87s/it]


In [ ]:
test_data['summary_results']=summ_results

In [ ]:
test_data.to_pickle('contriever/test_data_100_summ.pkl')

# STYLE ANALYSIS

##### Style Functions

In [ ]:
import collections as coll
import math
import pickle
import string
import numpy as np
from nltk.corpus import cmudict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

nltk.download('cmudict')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def syllable_count_Manual(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("e"):
                count -= 1
    if count == 0:
        count += 1
    return count


# ---------------------------------------------------------------------
# COUNTS NUMBER OF SYLLABLES

def syllable_count(word):
    global cmuDictionary
    d = cmuDictionary
    try:
        syl = [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    except:
        syl = syllable_count_Manual(word)
    return syl

    # ----------------------------------------------------------------------------


# removing stop words plus punctuation.
def Avg_wordLength(str):
    str.translate(string.punctuation)
    tokens = word_tokenize(str, language='english')
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    return np.average([len(word) for word in words])


# ----------------------------------------------------------------------------


# returns avg number of characters in a sentence
def Avg_SentLenghtByCh(text):
    tokens = sent_tokenize(text)
    return np.average([len(token) for token in tokens])


# ----------------------------------------------------------------------------

# returns avg number of words in a sentence
def Avg_SentLenghtByWord(text):
    tokens = sent_tokenize(text)
    return np.average([len(token.split()) for token in tokens])


# ----------------------------------------------------------------------------


# GIVES NUMBER OF SYLLABLES PER WORD
def Avg_Syllable_per_Word(text):
    tokens = word_tokenize(text, language='english')
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    syllabls = [syllable_count(word) for word in words]
    p = (" ".join(words))
    return sum(syllabls) / max(1, len(words))


# -----------------------------------------------------------------------------

# COUNTS SPECIAL CHARACTERS NORMALIZED OVER LENGTH OF CHUNK
def CountSpecialCharacter(text):
    st = ["#", "$", "%", "&", "(", ")", "*", "+", "-", "/", "<", "=", '>',
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    return count / len(text)


# ----------------------------------------------------------------------------

def CountPuncuation(text):
    st = [",", ".", "'", "!", '"', ";", "?", ":", ";"]
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    return float(count) / float(len(text))


# ----------------------------------------------------------------------------
# RETURNS NORMALIZED COUNT OF FUNCTIONAL WORDS FROM A Framework for
# Authorship Identification of Online Messages: Writing-Style Features and Classification Techniques

def CountFunctionalWords(text):
    functional_words = """a between in nor some upon
    about both including nothing somebody us
    above but inside of someone used
    after by into off something via
    all can is on such we
    although cos it once than what
    am do its one that whatever
    among down latter onto the when
    an each less opposite their where
    and either like or them whether
    another enough little our these which
    any every lots outside they while
    anybody everybody many over this who
    anyone everyone me own those whoever
    anything everything more past though whom
    are few most per through whose
    around following much plenty till will
    as for must plus to with
    at from my regarding toward within
    be have near same towards without
    because he need several under worth
    before her neither she unless would
    behind him no should unlike yes
    below i nobody since until you
    beside if none so up your
    """

    functional_words = functional_words.split()
    words = RemoveSpecialCHs(text)
    count = 0

    for i in text:
        if i in functional_words:
            count += 1

    return count / len(words)


# ---------------------------------------------------------------------------

# also returns Honore Measure R
def hapaxLegemena(text):
    words = RemoveSpecialCHs(text)
    V1 = 0
    # dictionary comprehension . har word kay against value 0 kardi
    freqs = {key: 0 for key in words}
    for word in words:
        freqs[word] += 1
    for word in freqs:
        if freqs[word] == 1:
            V1 += 1
    N = len(words)
    V = float(len(set(words)))
    R = 100 * math.log(N) / max(1, (1 - (V1 / V)))
    h = V1 / N
    return R, h


# ---------------------------------------------------------------------------

def hapaxDisLegemena(text):
    words = RemoveSpecialCHs(text)
    count = 0
    # Collections as coll Counter takes an iterable collapse duplicate and counts as
    # a dictionary how many equivelant items has been entered
    freqs = coll.Counter()
    freqs.update(words)
    for word in freqs:
        if freqs[word] == 2:
            count += 1

    h = count / float(len(words))
    S = count / float(len(set(words)))
    return S, h


# ---------------------------------------------------------------------------

# c(w)  = ceil (log2 (f(w*)/f(w))) f(w*) frequency of most commonly used words f(w) frequency of word w
# measure of vocabulary richness and connected to zipfs law, f(w*) const rak kay zips law say rank nikal rahay hein
def AvgWordFrequencyClass(text):
    words = RemoveSpecialCHs(text)
    # dictionary comprehension . har word kay against value 0 kardi
    freqs = {key: 0 for key in words}
    for word in words:
        freqs[word] += 1
    maximum = float(max(list(freqs.values())))
    return np.average([math.floor(math.log((maximum + 1) / (freqs[word]) + 1, 2)) for word in words])


# --------------------------------------------------------------------------
# TYPE TOKEN RATIO NO OF DIFFERENT WORDS / NO OF WORDS
def typeTokenRatio(text):
    words = word_tokenize(text)
    return len(set(words)) / len(words)


# --------------------------------------------------------------------------
# logW = V-a/log(N)
# N = total words , V = vocabulary richness (unique words) ,  a=0.17
# we can convert into log because we are only comparing different texts
def BrunetsMeasureW(text):
    words = RemoveSpecialCHs(text)
    a = 0.17
    V = float(len(set(words)))
    N = len(words)
    B = (V - a) / (math.log(N))
    return B


# ------------------------------------------------------------------------
def RemoveSpecialCHs(text):
    text = word_tokenize(text)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']

    words = [word for word in text if word not in st]
    return words


# -------------------------------------------------------------------------
# K  10,000 * (M - N) / N**2
# , where M  Sigma i**2 * Vi.
def YulesCharacteristicK(text):
    words = RemoveSpecialCHs(text)
    N = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    vi = coll.Counter()
    vi.update(freqs.values())
    M = sum([(value * value) * vi[value] for key, value in freqs.items()])
    K = 10000 * (M - N) / math.pow(N, 2)
    return K


# -------------------------------------------------------------------------


# -1*sigma(pi*lnpi)
# Shannon and sympsons index are basically diversity indices for any community
def ShannonEntropy(text):
    words = RemoveSpecialCHs(text)
    lenght = len(words)
    freqs = coll.Counter()
    freqs.update(words)
    arr = np.array(list(freqs.values()))
    distribution = 1. * arr
    distribution /= max(1, lenght)
    import scipy as sc
    H = sc.stats.entropy(distribution, base=2)
    # H = sum([(i/lenght)*math.log(i/lenght,math.e) for i in freqs.values()])
    return H


# ------------------------------------------------------------------
# 1 - (sigma(n(n - 1))/N(N-1)
# N is total number of words
# n is the number of each type of word
def SimpsonsIndex(text):
    words = RemoveSpecialCHs(text)
    freqs = coll.Counter()
    freqs.update(words)
    N = len(words)
    n = sum([1.0 * i * (i - 1) for i in freqs.values()])
    D = 1 - (n / (N * (N - 1)))
    return D


# ------------------------------------------------------------------

def FleschReadingEase(text, NoOfsentences):
    words = RemoveSpecialCHs(text)
    l = float(len(words))
    scount = 0
    for word in words:
        scount += syllable_count(word)

    I = 206.835 - 1.015 * (l / float(NoOfsentences)) - 84.6 * (scount / float(l))
    return I


# -------------------------------------------------------------------
def FleschCincadeGradeLevel(text, NoOfSentences):
    words = RemoveSpecialCHs(text)
    scount = 0
    for word in words:
        scount += syllable_count(word)

    l = len(words)
    F = 0.39 * (l / NoOfSentences) + 11.8 * (scount / float(l)) - 15.59
    return F


# -----------------------------------------------------------------
def dale_chall_readability_formula(text, NoOfSectences):
    words = RemoveSpecialCHs(text)
    difficult = 0
    adjusted = 0
    NoOfWords = len(words)
    with open('dale-chall.pkl', 'rb') as f:
        fimiliarWords = pickle.load(f)
    for word in words:
        if word not in fimiliarWords:
            difficult += 1
    percent = (difficult / NoOfWords) * 100
    if (percent > 5):
        adjusted = 3.6365
    D = 0.1579 * (percent) + 0.0496 * (NoOfWords / NoOfSectences) + adjusted
    return D


# ------------------------------------------------------------------
def GunningFoxIndex(text, NoOfSentences):
    words = RemoveSpecialCHs(text)
    NoOFWords = float(len(words))
    complexWords = 0
    for word in words:
        if (syllable_count(word) > 2):
            complexWords += 1

    G = 0.4 * ((NoOFWords / NoOfSentences) + 100 * (complexWords / NoOFWords))
    return G


# ------------------------------------------------------------------

#### Calls

In [ ]:
cmuDictionary = None
def FeatureExtraction(chunk, winSize):
    global cmuDictionary
    cmuDictionary = cmudict.dict()

    features = []

    # LEXICAL FEATURES
    mean_word_length = Avg_wordLength(chunk)
    features.append(f"Mean Word Length: {mean_word_length}")

    mean_sentence_length_chars = Avg_SentLenghtByCh(chunk)
    features.append(f"Mean Sentence Length (characters): {mean_sentence_length_chars}")

    mean_sentence_length_words = Avg_SentLenghtByWord(chunk)
    features.append(f"Mean Sentence Length (words): {mean_sentence_length_words}")

    mean_syllables_per_word = Avg_Syllable_per_Word(chunk)
    features.append(f"Mean Syllables per Word: {mean_syllables_per_word}")

    count_special_characters = CountSpecialCharacter(chunk)
    features.append(f"Count of Special Characters: {count_special_characters}")

    count_punctuation = CountPuncuation(chunk)
    features.append(f"Count of Punctuation: {count_punctuation}")

    count_functional_words = CountFunctionalWords(chunk)
    features.append(f"Count of Functional Words: {count_functional_words}")

    # VOCABULARY RICHNESS FEATURES
    type_token_ratio = typeTokenRatio(chunk)
    features.append(f"Type-Token Ratio: {type_token_ratio}")

    honore_measure_r, hapax_legemena = hapaxLegemena(chunk)
    features.append(f"Hapax Legemena: {hapax_legemena}, Honore Measure R: {honore_measure_r}")

    sicheles_measure_s, hapax_dis_legemena = hapaxDisLegemena(chunk)
    features.append(f"Hapax Dis Legemena: {hapax_dis_legemena}, Sicheles Measure S: {sicheles_measure_s}")

    yules_characteristic_k = YulesCharacteristicK(chunk)
    features.append(f"Yules Characteristic K: {yules_characteristic_k}")

    simpsons_index = SimpsonsIndex(chunk)
    features.append(f"Simpsons Index: {simpsons_index}")

    brunets_measure_w = BrunetsMeasureW(chunk)
    features.append(f"Brunets Measure W: {brunets_measure_w}")

    shannon_entropy = ShannonEntropy(chunk)
    features.append(f"Shannon Entropy: {shannon_entropy}")

    # READABILITY FEATURES
    flesch_reading_ease = FleschReadingEase(chunk, winSize)
    features.append(f"Flesch Reading Ease: {flesch_reading_ease}")

    flesch_cincade_grade_level = FleschCincadeGradeLevel(chunk, winSize)
    features.append(f"Flesch Cincade Grade Level: {flesch_cincade_grade_level}")

    #dale_chall_readability = dale_chall_readability_formula(chunk, winSize)
    #features.append(f"Dale Chall Readability Formula: {dale_chall_readability}")

    gunning_fox_index = GunningFoxIndex(chunk, winSize)
    features.append(f"Gunning Fox Index: {gunning_fox_index}")

    return '\n'.join(features)



def FeatureExtraction_titles(chunk_list):
    global cmuDictionary
    cmuDictionary = cmudict.dict()

    features = []

    # Initialize dictionaries to store sum and count for averaging
    sum_dict = {}
    count_dict = {}

    # Function to update sum and count dictionaries
    def update_dicts(feature_name, value):
        if feature_name not in sum_dict:
            sum_dict[feature_name] = value
            count_dict[feature_name] = 1
        else:
            sum_dict[feature_name] += value
            count_dict[feature_name] += 1

    # Iterate over each chunk in the list
    for chunk in chunk_list:
        # LEXICAL FEATURES
        update_dicts("Mean Word Length", Avg_wordLength(chunk))
        update_dicts("Mean Sentence Length (characters)", Avg_SentLenghtByCh(chunk))
        update_dicts("Mean Sentence Length (words)", Avg_SentLenghtByWord(chunk))
        update_dicts("Mean Syllables per Word", Avg_Syllable_per_Word(chunk))
        update_dicts("Count of Special Characters", CountSpecialCharacter(chunk))
        update_dicts("Count of Punctuation", CountPuncuation(chunk))
        update_dicts("Count of Functional Words", CountFunctionalWords(chunk))

        # VOCABULARY RICHNESS FEATURES
        update_dicts("Type-Token Ratio", typeTokenRatio(chunk))
        honore_measure_r, hapax_legemena = hapaxLegemena(chunk)
        update_dicts("Hapax Legemena", hapax_legemena)
        update_dicts("Honore Measure R", honore_measure_r)
        sicheles_measure_s, hapax_dis_legemena = hapaxDisLegemena(chunk)
        update_dicts("Hapax Dis Legemena", hapax_dis_legemena)
        update_dicts("Sicheles Measure S", sicheles_measure_s)
        update_dicts("Yules Characteristic K", YulesCharacteristicK(chunk))
        update_dicts("Simpsons Index", SimpsonsIndex(chunk))
        update_dicts("Brunets Measure W", BrunetsMeasureW(chunk))
        update_dicts("Shannon Entropy", ShannonEntropy(chunk))

        # READABILITY FEATURES
        update_dicts("Flesch Reading Ease", FleschReadingEase(chunk, len(chunk)))
        update_dicts("Flesch Cincade Grade Level", FleschCincadeGradeLevel(chunk,len(chunk)))
        #update_dicts("Dale Chall Readability Formula", dale_chall_readability_formula(chunk,len(chunk)))
        update_dicts("Gunning Fox Index", GunningFoxIndex(chunk, len(chunk)))

    # Calculate the average for each feature
    for feature_name, feature_sum in sum_dict.items():
        avg_value = feature_sum / count_dict[feature_name]
        features.append(f"{feature_name}: {avg_value}")

    return '\n'.join(features)

#### Test

In [ ]:
exp='''
Lexical Features:
1. Mean Word Length: Average number of characters in words.
2. Mean Sentence Length (characters): Average number of characters in sentences.
3. Mean Sentence Length (words): Average number of words in sentences.
4. Mean Syllables per Word: Average number of syllables in words.
5. Count of Special Characters: Normalized count of special characters over the length of the chunk.
6. Count of Punctuation: Normalized count of punctuation marks over the length of the chunk.
7. Count of Functional Words: Normalized count of functional words from a specific set.

Vocabulary Richness Features:
8. Type-Token Ratio: Ratio of different words to total words.
9. Hapax Legemena: Ratio of words that occur only once to total words.
10. Sicheles Measure S: Ratio of words that occur twice to total words.
11. Yules Characteristic K: Measure of vocabulary richness and connection to Zipf's Law.
12. Simpsons Index: Diversity index based on word frequency.
13. Brunets Measure W: Vocabulary richness index.
14. Shannon Entropy: Information entropy based on word frequency.

Readability Features:
15. Flesch Reading Ease: Readability score based on average sentence length and syllables per word.
16. Flesch Cincade Grade Level: Readability score based on average sentence length and syllables per word.
17. Dale Chall Readability Formula: Readability score considering difficult words.
18. Gunning Fox Index: Readability index based on sentence length and complex words.

'''

#### LLM BASED

In [ ]:
## PROMPT for LLM BASE WRITING STYLE

In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as palm
import os
os.environ['API_KEY']=#API KEY HERE
palm.configure(api_key=os.environ['API_KEY'])

## RUN STYLE ANALYSIS ON TEST DATA

In [ ]:
import time

In [ ]:
def get_palm_response(input_text, prompt_instructions):
  p="input text: "+ input_text+"\n"+prompt_instructions
  response = palm.generate_text(prompt=p)
  return response.result

In [ ]:
def get_writing_style(text,prompt_instructions,all_titles):
  statistical_analysis= FeatureExtraction_titles(all_titles)
  llm_bases_analysis=get_palm_response(text, prompt_instructions)

  return {'statistical_analysis':statistical_analysis,'llm_bases_analysis':llm_bases_analysis }

In [ ]:
prompt_instructions='''You are a writing style detector. Your job is to specify a writing style of input text above. When you do so, you describe outloud:
general writing style (like creative, non-fiction),
genres,
style descriptors (like full of dialogue, short sentences),
moods (like serious, joking),
authors that share a similar style (like Kurt Vonnegut),
common phrases or speech patterns. You are not to provide any other additional information.
'''
style_results=[]
for row in tqdm(range(test_data.shape[0])):
  cur_list = test_data.iloc[row]['summary_results']

  all_titles = [entry['title'] for entry in cur_list]
  all_profile_article = [entry['title']+'\n'+entry['text'] for entry in cur_list]

  style=get_writing_style("\n".join(all_profile_article),prompt_instructions,all_titles)
  style_results.append(style)

  if(row%10==0):
    # Save to a pickle file
    with open("style_checkpoints/cont_style_summ"+str(row)+".pkl", "wb") as file:
      pickle.dump(style_results, file)
      time.sleep(30)

100%|██████████| 100/100 [09:52<00:00,  5.93s/it]


In [ ]:
test_data['style_results']=style_results

In [ ]:
test_data.to_pickle('contriever/test_data_100_style.pkl')

In [ ]:
test_data.head(1)

,id,input,profile,output,bm25_results,keyword_results,summary_results,style_results
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,[{'abstract': 'Industrial Control Systems (ICS...,[{'text': 'Industrial Control Systems (ICS) ar...,[{'text': 'Industrial Control Systems (ICS) ar...,{'statistical_analysis': 'Mean Word Length: 8....


In [ ]:
test_data.iloc[0]['input']

"Generate a title for the following abstract of a paper: Industrial Control Systems (ICS) such as water and power are critical to any society. Process anomaly detection mechanisms have been proposed to protect such systems to minimize the risk of damage or loss of resources. In this paper, a graphical model-based approach is proposed for profiling normal operational behavior of an operational ICS referred to as SWaT (Secure Water Treatment). Timed automata are learned as a model of regular behaviors shown in sensors signal like fluctuations of water level in tanks. Bayesian networks are learned to discover dependencies between sensors and actuators. The models are used as a one-class classifier for process anomaly detection, recognizing irregular behavioral patterns and dependencies. The detection results can be interpreted and the abnormal sensors or actuators localized due to the interpretability of the graphical models. This approach is applied to a dataset collected from SWaT. Expe

## Adding keywords and summary for input too.

In [ ]:
keyword_list=[]
summary_list=[]
for row in tqdm(range(test_data.shape[0])):
  input = test_data.iloc[row]['input'].split("Generate a title for the following abstract of a paper:")[-1].strip()
  keywords_scores=kw_model.extract_keywords(input)
  keyword_list.append(', '.join([kw[0] for kw in keywords_scores]))
  summary_list.append(get_summ(input))


In [ ]:
prev=pd.read_pickle('bm25/preprompt_100.pkl')
keyword_list=prev['input_keywords']
summary_list=prev["input_summary"]

In [ ]:
test_data['input_keywords']=keyword_list
test_data["input_summary"]=summary_list

In [ ]:
test_data.to_pickle('contriever/preprompt_100.pkl')

# COMPLETE PIPELINE

In [ ]:
preprompt_data=pd.read_pickle("contriever/preprompt_100.pkl")

In [ ]:
preprompt_data.head(1)

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results,summary_results,style_results
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,"automata, swat, classifier, detection, anomaly",the problem of process detection in water tank...,[{'title': 'WaterJam: An Experimental Case Stu...,[{'text': 'An experiment was conducted to inve...,[{'text': 'An experiment was conducted to inve...,{'statistical_analysis': 'Mean Word Length: 7....


In [ ]:
def generate_title_prompt(input_text, style_results, summary_results,ip_keys,ip_summ):
    # Extracting style analysis and general writing style of the user
    style_analysis = style_results['statistical_analysis']
    general_style = style_results['llm_bases_analysis']

    # Extracting information from previous articles
    previous_article_prompts = []
    for article in summary_results:
        article_text = article['text']
        title = article['title']
        keywords = ', '.join([kw[0] for kw in article['keywords']])
        summary = article['summary']
        previous_article_prompt = (
            f"Abstract Text: {article_text}\n"
            f"Title: {title}\n"
            f"Keywords: {keywords}\n"
            f"Summary: {summary}\n\n"
        )
        previous_article_prompts.append(previous_article_prompt)

    # Combining all information into the final prompt
    final_prompt_details = (
        f"Previous Articles Analysis:\n{''.join(previous_article_prompts)}"
        f"Statistical Writing Style Analysis of Titles:\n{style_analysis}\n\n"
        f"General Writing Style of User:\n{general_style}\n\n"
        f"New Input Article:\nAbstract Text: {input_text}\n"
        f"Keywords: {ip_keys}\n"
        f"Summary: {ip_summ}\n\n"
    )

    final_prompt="Your task is to analyze the provided data to understand the user's writing style and approach to titling. This includes examining previous articles with their titles, keywords, and summaries, a statistical analysis of the user's writing style in their titles, a general characterization of their writing style, and the abstract of a new input article. Based on this analysis, generate a title for the new article that aligns with the user's unique style."\
    +"\n"\
    +final_prompt_details\
    +"\n"+"After reviewing all these elements, create a title for the new article that reflects the user's unique writing style and titling approach, as well as being pertinent to the new article's content and theme"

    return final_prompt


In [ ]:
# Example usage
input_article = preprompt_data.iloc[0]['input']
ip_keys=preprompt_data.iloc[0]['input_keywords']
ip_summ=preprompt_data.iloc[0]['input_summary']
style_results = preprompt_data.iloc[0]['style_results']
summary_results = preprompt_data.iloc[0]['summary_results']

prompt_to_llm = generate_title_prompt(input_article, style_results, summary_results,ip_keys,ip_summ)
pprint(prompt_to_llm)

("Your task is to analyze the provided data to understand the user's writing "
 'style and approach to titling. This includes examining previous articles '
 'with their titles, keywords, and summaries, a statistical analysis of the '
 "user's writing style in their titles, a general characterization of their "
 'writing style, and the abstract of a new input article. Based on this '
 "analysis, generate a title for the new article that aligns with the user's "
 'unique style.\n'
 'Previous Articles Analysis:\n'
 'Abstract Text: An experiment was conducted to investigate network jamming '
 'attacks on an Industrial Control Systems. The Secure Water Treatment (SWaT) '
 'system was chosen to perform the experiments. Jamming attacks were launched '
 'on SWaT using software defined radio. Attacks were designed to meet attacker '
 'objectives selected from a Cyber Physical Systems specific attacker model. '
 'Attacks exposed vulnerabilities associated with design of SWaT and suggested '
 'mo

In [ ]:
prompt_list=[]
for row in tqdm(range(preprompt_data.shape[0])):
  input_article = preprompt_data.iloc[row]['input']
  ip_keys=preprompt_data.iloc[row]['input_keywords']
  ip_summ=preprompt_data.iloc[row]['input_summary']
  style_results = preprompt_data.iloc[row]['style_results']
  summary_results = preprompt_data.iloc[row]['summary_results']
  prompt_list.append(generate_title_prompt(input_article, style_results, summary_results,ip_keys,ip_summ))


100%|██████████| 100/100 [00:00<00:00, 1999.47it/s]


In [ ]:
preprompt_data['prompts']=prompt_list
preprompt_data.to_pickle('contriever/preprompt_100.pkl')

# RUN PROMPTS THROUGH PALM

In [ ]:
preprompt_data=pd.read_pickle('contriever/preprompt_100.pkl')

In [ ]:
preprompt_data.head(1)

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results,summary_results,style_results,prompts
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,"automata, swat, classifier, detection, anomaly",the problem of process detection in water tank...,[{'title': 'WaterJam: An Experimental Case Stu...,[{'text': 'An experiment was conducted to inve...,[{'text': 'An experiment was conducted to inve...,{'statistical_analysis': 'Mean Word Length: 7....,Your task is to analyze the provided data to u...


In [ ]:
import pickle
import time
response_list=[]
for row in tqdm(range(preprompt_data.shape[0])):
  cur_prompt=preprompt_data.iloc[row]['prompts']
  response = palm.generate_text(prompt=cur_prompt)
  response_list.append(response.result)

  if(row%10==0):
    # Save to a pickle file
    with open("prompting_checkpoints/cont_prompt_out"+str(row)+".pkl", "wb") as file:
      pickle.dump(response_list, file)
      time.sleep(30)


preprompt_data['model_output']=response_list
preprompt_data.to_pickle('contriever/postprompt_100.pkl')

100%|██████████| 100/100 [08:35<00:00,  5.16s/it]


In [ ]:
preprompt_data.head(1)

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results,summary_results,style_results,prompts,model_output
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,"automata, swat, classifier, detection, anomaly",the problem of process detection in water tank...,[{'title': 'WaterJam: An Experimental Case Stu...,[{'text': 'An experiment was conducted to inve...,[{'text': 'An experiment was conducted to inve...,{'statistical_analysis': 'Mean Word Length: 7....,Your task is to analyze the provided data to u...,A Graphical Model-Based Approach for Anomaly D...


## GETTING NO PERSONALIZED RESULTS

In [ ]:
post_prompt_data=pd.read_pickle('postprompt_100.pkl')

In [ ]:
print(post_prompt_data.iloc[2]['prompts'])

In [ ]:
import pickle
import time
response_list=[]
for row in tqdm(range(post_prompt_data.shape[0])):
  response = palm.generate_text(prompt=post_prompt_data.iloc[row]['input'])
  response_list.append(response.result)

  if(row%10==0):
    time.sleep(20)

100%|██████████| 100/100 [09:20<00:00,  5.60s/it]


In [ ]:
len(response_list)

100

In [ ]:
post_prompt_data['no_per_model_output']=response_list

In [ ]:
post_prompt_data.head(1)

,id,input,profile,output,bm25_results,keyword_results,summary_results,style_results,input_keywords,input_summary,prompts,model_output,no_per_model_output
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,[{'abstract': 'Industrial Control Systems (ICS...,[{'text': 'Industrial Control Systems (ICS) ar...,[{'text': 'Industrial Control Systems (ICS) ar...,{'statistical_analysis': 'Mean Word Length: 8....,"automata, swat, classifier, detection, anomaly",the problem of process detection in water tank...,Your task is to analyze the provided data to u...,A Graphical Model-Based Approach for Process A...,Learning Graphical Models for Process Anomaly ...


In [ ]:
post_prompt_data.to_pickle('postprompt_100.pkl')

# Evaluation

In [ ]:
!pip install rouge

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from rouge import Rouge

In [ ]:
def getRougeScores(groundTruth, generatedPrompt):
  rouge = Rouge()
  scores = rouge.get_scores(text1, text2)
  r1, r2, rl = scores[0]['rouge-1']['r'] , scores[0]['rouge-2']['r'] , scores[0]['rouge-l']['r']

  return r1,r2,rl

###############################

def getBertScore(groundTruth, generatedPrompt):
  tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
  model = BertModel.from_pretrained("bert-base-uncased")

  # Step 1: Prepare the texts for BERT
  inputs1 = tokenizer(groundTruth, return_tensors="pt", padding=True, truncation=True)
  inputs2 = tokenizer(generatedPrompt, return_tensors="pt", padding=True, truncation=True)

  # Step 2: Feed the texts to the BERT model
  outputs1 = model(**inputs1)
  outputs2 = model(**inputs2)

  # Step 3: Obtain the representation vectors
  embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
  embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()

  # Step 4: Calculate cosine similarity
  similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))

  return similarity[0][0]

In [ ]:
import pandas as pd
import pickle as pk

from pandas import DataFrame

In [ ]:
retriever_results = pd.read_pickle(r'contriever/postprompt_100.pkl')

retriever_results.head()

,id,input,profile,output,input_keywords,input_summary,contriever,keyword_results,summary_results,style_results,prompts,model_output
8913,408913,Generate a title for the following abstract of...,[{'title': 'Fault domain-based testing in impe...,TABOR: A Graphical Model-based Approach for An...,"automata, swat, classifier, detection, anomaly",the problem of process detection in water tank...,[{'title': 'WaterJam: An Experimental Case Stu...,[{'text': 'An experiment was conducted to inve...,[{'text': 'An experiment was conducted to inve...,{'statistical_analysis': 'Mean Word Length: 7....,Your task is to analyze the provided data to u...,A Graphical Model-Based Approach for Anomaly D...
2242,402242,Generate a title for the following abstract of...,"[{'title': 'On updating signal subspaces', 'ab...",Hands-On Training for Undergraduates in High-P...,"java, programming, computing, computational, p...",this paper describes a practical computing cou...,[{'title': 'Parallel programming in computatio...,[{'text': 'Parallel programming of high-perfor...,[{'text': 'Parallel programming of high-perfor...,{'statistical_analysis': 'Mean Word Length: 8....,Your task is to analyze the provided data to u...,A Practical Training Course in High-Performanc...
2577,402577,Generate a title for the following abstract of...,[{'title': 'A Note on Filtration and Granular ...,On possible rules and apriori algorithm in non...,"algorithm, minimalcertainrules, deterministici...","in this paper, we introduce a framework of gen...",[{'title': 'Discernibility functions and minim...,[{'text': 'Minimal rule generation in Non-dete...,[{'text': 'Minimal rule generation in Non-dete...,{'statistical_analysis': 'Mean Word Length: 8....,Your task is to analyze the provided data to u...,Min-Max and Max-Max Strategies for Possible Ru...
1399,401399,Generate a title for the following abstract of...,[{'title': 'Temporal representation for scient...,Framework for bringing data streams to the grid,"streams, stream, streaming, grid, data",this article introduces a model for bringing e...,[{'title': 'Calder Query Grid Service: Insight...,[{'text': 'We have architected and evaluated a...,[{'text': 'We have architected and evaluated a...,{'statistical_analysis': 'Mean Word Length: 7....,Your task is to analyze the provided data to u...,A Model for Bringing Existing Data Stream Syst...
7707,407707,Generate a title for the following abstract of...,[{'title': 'Completeness In Differential Appro...,Multiparameterizations for max $k$-set cover a...,"parameterizations, complexity, parameterized, ...",study the complexity of several parameterizati...,[{'title': 'Multi-parameter Analysis for Local...,[{'text': 'We study the parameterized complexi...,[{'text': 'We study the parameterized complexi...,{'statistical_analysis': 'Mean Word Length: 8....,Your task is to analyze the provided data to u...,Multiparameterized complexity of max k-set cov...


In [ ]:
bertScore=[]
for index, row in retriever_results.iterrows() :
  output = row['output']
  modelOp = row['model_output']

  if output != "" and output != None and modelOp != "" and modelOp != None :
    bertScore.append(getBertScore(output, modelOp))
  else :
    bertScore.append(0)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
r1lst = []
r2lst = []
rllst = []

rouge = Rouge()

for index, row in retriever_results.iterrows() :
  output = row['output']
  modelOp = row['model_output']

  if output != "" and output != None and modelOp != "" and modelOp != None :
    scores = rouge.get_scores(output, modelOp)
    r1, r2, rl = scores[0]['rouge-1']['r'] , scores[0]['rouge-2']['r'] , scores[0]['rouge-l']['r']
  else:
    r1, r2, rl = 0,0,0

  r1lst.append(r1)
  r2lst.append(r2)
  rllst.append(rl)

In [ ]:
retriever_results['BertScore-P'] = bertScore
retriever_results['r1-rouge-P'] = r1lst
retriever_results['r2-rouge-P'] = r2lst
retriever_results['rl-rouge-P'] = rllst

In [ ]:
retriever_results.to_pickle("contriever/metric_output.pkl")

In [ ]:
print("Mean r1-rouge {}".format(retriever_results['r1-rouge-P'].mean()))
print("Mean r2-rouge {}".format(retriever_results['r2-rouge-P'].mean()))
print("Mean rl-rouge {}".format(retriever_results['rl-rouge-P'].mean()))
print("Mean bert {}".format(retriever_results['BertScore-P'].mean()))

Mean r1-rouge 0.2759768483860556
Mean r2-rouge 0.12609967075408252
Mean rl-rouge 0.2537378096470168
Mean bert 0.8370622849464416


In [ ]:
bertSum = 0
r1Sum = 0
r2Sum = 0
NoneCount = 0
r = 0

for index, row in retriever_results.iterrows() :
  if row['model_output'] == None:
    NoneCount += 1
  else:
    bertSum += row['BertScore-P']
    r1Sum += row['r1-rouge-P']
    r2Sum += row['r2-rouge-P']
    r += row['rl-rouge-P']

noneNull = retriever_results.shape[0] - NoneCount

print("Mean r1", r1Sum/noneNull)
print("Mean r2", r2Sum/noneNull)
print("Mean rl", r/noneNull)
print("Mean bert", bertSum/noneNull)
print("None Count: ", NoneCount)

Mean r1 0.28747588373547456
Mean r2 0.1313538237021693
Mean rl 0.2643102183823091
Mean bert 0.8719398801525434
None Count:  4


In [ ]:
bertScore=[]
for index, row in retriever_results.iterrows() :
  output = row['output']
  modelOp = row['no_per_model_output']

  if output != "" and output != None and modelOp != "" and modelOp != None :
    bertScore.append(getBertScore(output, modelOp))
  else :
    bertScore.append(0)

r1lst = []
r2lst = []
rllst = []

rouge = Rouge()

for index, row in retriever_results.iterrows() :
  output = row['output']
  modelOp = row['no_per_model_output']

  if output != "" and output != None and modelOp != "" and modelOp != None :
    scores = rouge.get_scores(output, modelOp)
    r1, r2, rl = scores[0]['rouge-1']['r'] , scores[0]['rouge-2']['r'] , scores[0]['rouge-l']['r']
  else:
    r1, r2, rl = 0,0,0

  r1lst.append(r1)
  r2lst.append(r2)
  rllst.append(rl)

bertSum = 0
r1Sum = 0
r2Sum = 0
NoneCount = 0
r = 0


retriever_results['BertScore-NP'] = bertScore
retriever_results['r1-rouge-NP'] = r1lst
retriever_results['r2-rouge-NP'] = r2lst
retriever_results['rl-rouge-NP'] = rllst


bertSum = 0
r1Sum = 0
r2Sum = 0
NoneCount = 0
r = 0

for index, row in retriever_results.iterrows() :
  if row['no_per_model_output'] == None:
    NoneCount += 1
  else:
    bertSum += row['BertScore-NP']
    r1Sum += row['r1-rouge-NP']
    r2Sum += row['r2-rouge-NP']
    r += row['rl-rouge-NP']

noneNull = retriever_results.shape[0] - NoneCount

print("Mean r1", r1Sum/noneNull)
print("Mean r2", r2Sum/noneNull)
print("Mean rl", r/noneNull)
print("Mean bert", bertSum/noneNull)
print("None Count: ", NoneCount)

In [ ]:
retriever_results.to_pickle("metric_output.pkl")